# Is there still room for a new pizzeria in Rome, Italy?

*A data study by Charlotte Marjolein Baas, MSc*

## Introduction

Having a restaurant in Rome, Italy is one of the biggest dreams of a pizza maker. However, starting one might not be the biggest challenge. In a place like Rome there are assumed to be pizzerias on about every corner of every busy street. The competition is huge if not killing or completely murderous.

I have been contacted by a pizza lover who wants to know if (and if so, then where) there is still room for a new pizzeria in the city of Rome. This pizza lover is aching to start his own pizza place in the best part of the city, however, he knows there is a lot of competition. He asked me to investigate where he stands the most chance of getting his new restaurant running, i.e. where the competition is the least ‘killing’ and people are still willing to try out his pizzas.


## Data

In order to answer the main question: Is there still room for a new pizzeria in the city of Rome, Italy, subdivisions (municipio) of Rome were scraped from wikipedia.org (https://en.wikipedia.org/wiki/Administrative_subdivision_of_Rome). This scraping resulted in a dataframe containing the names of the fifteen municipio, their corresponding population, area in square kilometers and density in square kilometers. In addition, the coordinates of the different municipio were collected from www.geonames.org. A csv-file was manually created with the corresponding latitude and longitude coordinates of each municipio (all were positioned on different pages on the site).

Data was further pre-processed merging both the dataframe containing the information from wikipedia.org and the manually created csv-file with information from geonames.org.

The municipio of Rome were labeled on a map, showing their positions in the city.

With the use of the Foursquare API the venues of the fifteen municipio were explored per subdivision (within a range of 1500 meters from the coordinates of the subdivision, with a limit of 100 venues). The resulting venues were filtered, leaving anything out except for the categories 'pizza place' and 'Italian restaurant'. For every subdivision of the city, this resulted in a dataframe containing the pizza places and Italian restaurants.

In order to come up with a meaningful answer to the main question, the number of relevant venues in each municipio were combined into a new dataframe. The population density was taken into account during the search for the best place for a new pizzeria. Municipio containing the lowest 'competitors' in combination with the highest population density were considered to be the 'place to be' for a new pizzeria.


## Methodology

Download and import all dependencies before getting the data and exploring it

In [1]:
import numpy as np
import pandas as pd
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes

import folium
print("Libraries imported")




Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported


### Step 1: Explore venues of each municipio

Scrape Wikipedia site and transform data into pandas dataframe

In [2]:
url = 'https://en.wikipedia.org/wiki/Administrative_subdivision_of_Rome'
data = pd.read_html(url, header=0)
df_data = (pd.DataFrame.from_dict(data[0]))
df_data = df_data.drop("Map", axis=1)
df_data

,Municipio,Population 31 December 2015,Area in km²,Density per km²
0,Municipio I – Historical Center,186802,19.91,9382
1,Municipio II – Parioli/Nomentano,167736,19.60,8567
2,Municipio III – Monte Sacro,204514,97.82,2091
3,Municipio IV – Tiburtina,177084,49.15,3603
4,Municipio V – Prenestino/Centocelle,246471,27.00,9137
5,Municipio VI – Roma Delle Torri,256261,113.40,2261
6,Municipio VII – Appio-Latino/Tuscolano/Cinecittà,307607,46.80,6580
7,Municipio VIII – Appia Antica,131082,47.29,2772
8,Municipio IX – EUR,180511,183.17,985
9,Municipio X – Ostia/Acilia,230544,150.64,1530


Rename municipio and store municipio numbers in new column

In [3]:
new = df_data["Municipio"].str.split(" – ", n = 1, expand = True)
#new
new_1 = new[0].str.split("Municipio ",n=1, expand = True)
#new,new_1
new_2 = new_1[1]
df_data["Neighborhood"] = new[1]
df_data["Municipio"] = new_2
df_data.rename(columns={"Population 31 December 2015": "Population", "Area in km²": "Area", "Density per km²": "Density"}, inplace = True)
cols = ["Municipio", "Neighborhood", "Population", "Area", "Density"]
df_data = df_data[cols]
df_data

,Municipio,Neighborhood,Population,Area,Density
0,I,Historical Center,186802,19.91,9382
1,II,Parioli/Nomentano,167736,19.60,8567
2,III,Monte Sacro,204514,97.82,2091
3,IV,Tiburtina,177084,49.15,3603
4,V,Prenestino/Centocelle,246471,27.00,9137
5,VI,Roma Delle Torri,256261,113.40,2261
6,VII,Appio-Latino/Tuscolano/Cinecittà,307607,46.80,6580
7,VIII,Appia Antica,131082,47.29,2772
8,IX,EUR,180511,183.17,985
9,X,Ostia/Acilia,230544,150.64,1530


Get the latitude and longitude coordinates of each Municipio from csv file committed to github

In [4]:
geo_url = 'https://raw.githubusercontent.com/Baasje85/Coursera_Capstone/master/Geodata_Roma.csv'
geo_data = pd.read_csv(geo_url, index_col=0)
geo_data = geo_data.reset_index()
geo_data

,Municipio,Neighborhood,Latitude,Longitude
0,I,Historical Center,41.89798,12.47813
1,II,Parioli/Nomentano,41.92006,12.50136
2,III,Monte Sacro,41.99579,12.55450
3,IV,Tiburtina,41.93151,12.59152
4,V,Prenestino/Centocelle,41.88846,12.57574
5,VI,Roma Delle Torri,41.88690,12.68729
6,VII,Appio-Latino/Tuscolano/Cinecittà,41.83840,12.58207
7,VIII,Appia Antica,41.82802,12.52934
8,IX,EUR,41.75700,12.49799
9,X,Ostia/Acilia,41.73875,12.36328


In [5]:
merged_data = pd.merge(df_data, geo_data)
merged_data

,Municipio,Neighborhood,Population,Area,Density,Latitude,Longitude
0,I,Historical Center,186802,19.91,9382,41.89798,12.47813
1,II,Parioli/Nomentano,167736,19.60,8567,41.92006,12.50136
2,III,Monte Sacro,204514,97.82,2091,41.99579,12.55450
3,IV,Tiburtina,177084,49.15,3603,41.93151,12.59152
4,V,Prenestino/Centocelle,246471,27.00,9137,41.88846,12.57574
5,VI,Roma Delle Torri,256261,113.40,2261,41.88690,12.68729
6,VII,Appio-Latino/Tuscolano/Cinecittà,307607,46.80,6580,41.83840,12.58207
7,VIII,Appia Antica,131082,47.29,2772,41.82802,12.52934
8,IX,EUR,180511,183.17,985,41.75700,12.49799
9,X,Ostia/Acilia,230544,150.64,1530,41.73875,12.36328


*population on december 31st, 2015; area measured in square km; density measured per square km

Explore the neighborhoods in Rome

In [6]:
latitude = 41.902782
longitude = 12.496366

map_rome = folium.Map(location=[latitude, longitude], zoom_start = 10)
for lat, lng, municipio, neighborhood in zip(merged_data["Latitude"], merged_data["Longitude"], merged_data["Municipio"], merged_data["Neighborhood"]):
    label = '{},{}'.format(municipio, neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.Circle(
    [lat,lng],
    radius = 1500,
    popup=label,
    color = 'red',
    fill = True,
    fill_color = '#d98880',
    fill_opacity=0.7,
    parse_html = False).add_to(map_rome)

map_rome

Define Foursquare and explore first neighborhood in dataframe

In [7]:

CLIENT_ID = 'ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ' # your Foursquare ID
CLIENT_SECRET = 'Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

merged_data.loc[0, 'Neighborhood']

'Historical Center'

In [8]:
neighborhood_latitude_I = merged_data.loc[0, 'Latitude']
neighborhood_longitude_I = merged_data.loc[0, 'Longitude']
neighborhood_name_I = merged_data.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_I, neighborhood_latitude_I, neighborhood_longitude_I))

Latitude and longitude values of Historical Center are 41.89798, 12.47813


Get top 100 venues in **Historical Center** within radius of 1500 meters

In [9]:
LIMIT = 100
radius = 1500
url_I = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_I, 
    neighborhood_longitude_I, 
    radius, 
    LIMIT)
url_I

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.89798,12.47813&radius=1500&limit=100'

In [10]:
results_I = requests.get(url_I).json()

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues_I = results_I['response']['groups'][0]['items']
    
nearby_venues_I = json_normalize(venues_I) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_I =nearby_venues_I.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_I['venue.categories'] = nearby_venues_I.apply(get_category_type, axis=1)

# clean columns
nearby_venues_I.columns = [col.split(".")[-1] for col in nearby_venues_I.columns]

nearby_venues_I.head()



,name,categories,lat,lng
0,Pantheon,Monument / Landmark,41.899133,12.476805
1,Piazza della Rotonda,Plaza,41.899253,12.476779
2,Pizza e Mozzarella,Pizza Place,41.897598,12.479097
3,Il Panino Ingegnoso,Sandwich Place,41.899982,12.479195
4,Bartolucci,Toy / Game Store,41.899820,12.478812


Search for only Pizza places or Italian restaurants

In [13]:
Venues_I = nearby_venues_I.loc[nearby_venues_I['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_I

,name,categories,lat,lng
2,Pizza e Mozzarella,Pizza Place,41.897598,12.479097
15,Vini & Cucina,Pizza Place,41.896840,12.476409
17,Capranica Enoteca & Taverna,Italian Restaurant,41.899989,12.477762
32,Pizzeria Florida,Pizza Place,41.894819,12.476838
36,Emma,Pizza Place,41.894758,12.475082
40,La Tavernetta 48,Italian Restaurant,41.901029,12.475602
42,Hostaria Costanza,Italian Restaurant,41.896037,12.473708
50,Cantina e Cucina,Italian Restaurant,41.898002,12.471657
52,Pietro Valentini Ristorante,Italian Restaurant,41.900083,12.473632
55,Pizzeria da Pasquale,Pizza Place,41.902434,12.476703


Find second neighborhood

In [14]:
neighborhood_latitude_II = merged_data.loc[1, 'Latitude']
neighborhood_longitude_II = merged_data.loc[1, 'Longitude']
neighborhood_name_II = merged_data.loc[1, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_II, neighborhood_latitude_II, neighborhood_longitude_II))

Latitude and longitude values of Parioli/Nomentano are 41.92006, 12.50136


Get top 100 venues in **Parioli/Nomentano** within radius of 1500 meters

In [15]:
LIMIT = 100
radius = 1500
url_II = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_II, 
    neighborhood_longitude_II, 
    radius, 
    LIMIT)
url_II

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.92006,12.50136&radius=1500&limit=100'

In [16]:
results_II = requests.get(url_II).json()

In [17]:
venues_II = results_II['response']['groups'][0]['items']
    
nearby_venues_II = json_normalize(venues_II) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_II =nearby_venues_II.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_II['venue.categories'] = nearby_venues_II.apply(get_category_type, axis=1)

# clean columns
nearby_venues_II.columns = [col.split(".")[-1] for col in nearby_venues_II.columns]

nearby_venues_II.head()



,name,categories,lat,lng
0,Kilo Restaurant,Steakhouse,41.918245,12.498625
1,Bucavino,Italian Restaurant,41.917442,12.499237
2,Ristorante Lin,Chinese Restaurant,41.916750,12.499490
3,Hotel Buenos Aires Rome,Hotel,41.921268,12.502261
4,Mamma Mia,Italian Restaurant,41.918150,12.496798


Search for only Pizza places or Italian restaurants

In [18]:
Venues_II = nearby_venues_II.loc[nearby_venues_II['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_II

,name,categories,lat,lng
1,Bucavino,Italian Restaurant,41.917442,12.499237
4,Mamma Mia,Italian Restaurant,41.918150,12.496798
5,Pizzeria Da Agostino,Pizza Place,41.921787,12.507020
17,Taverna Rossini,Italian Restaurant,41.921848,12.492326
20,Il Gatto e L'uva,Italian Restaurant,41.911936,12.499929
22,Marzapane,Italian Restaurant,41.912543,12.498587
28,Al Forno della Soffitta,Italian Restaurant,41.909669,12.498445
29,Joe Padella,Italian Restaurant,41.913464,12.507268
30,Osteria dell'Arco,Italian Restaurant,41.910788,12.500321
31,Alice Pizza,Pizza Place,41.927205,12.492726


Find third neighborhood

In [19]:
neighborhood_latitude_III = merged_data.loc[2, 'Latitude']
neighborhood_longitude_III = merged_data.loc[2, 'Longitude']
neighborhood_name_III = merged_data.loc[2, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_III, neighborhood_latitude_III, neighborhood_longitude_III))

Latitude and longitude values of Monte Sacro are 41.99579, 12.5545


Get top 100 venues in **Monte Sacro** within radius of 1500 meters

In [20]:
LIMIT = 100
radius = 1500
url_III = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_III, 
    neighborhood_longitude_III, 
    radius, 
    LIMIT)
url_III

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.99579,12.5545&radius=1500&limit=100'

In [21]:
results_III = requests.get(url_III).json()

In [22]:
venues_III = results_III['response']['groups'][0]['items']
    
nearby_venues_III = json_normalize(venues_III) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_III =nearby_venues_III.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_III['venue.categories'] = nearby_venues_III.apply(get_category_type, axis=1)

# clean columns
nearby_venues_III.columns = [col.split(".")[-1] for col in nearby_venues_III.columns]

nearby_venues_III.head()



,name,categories,lat,lng
0,Circolo Valentini,Pool,41.990850,12.552933
1,Boma Countryhouse,Hotel,41.995153,12.563933
2,Azienda Agricola Fortunato,Farm,42.000633,12.564436
3,3Store,Mobile Phone Shop,41.986914,12.564226


Search for only Pizza places or Italian restaurants

In [23]:
Venues_III = nearby_venues_III.loc[nearby_venues_III['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_III

,name,categories,lat,lng


Find fourth neighborhood

In [24]:
neighborhood_latitude_IV = merged_data.loc[3, 'Latitude']
neighborhood_longitude_IV = merged_data.loc[3, 'Longitude']
neighborhood_name_IV = merged_data.loc[3, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_IV, neighborhood_latitude_IV, neighborhood_longitude_IV))

Latitude and longitude values of Tiburtina are 41.931509999999996, 12.59152


Get top 100 venues in **Tiburtina** within radius of 1500 meters

In [25]:
LIMIT = 100
radius = 1500
url_IV = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_IV, 
    neighborhood_longitude_IV, 
    radius, 
    LIMIT)
url_IV

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.931509999999996,12.59152&radius=1500&limit=100'

In [26]:
results_IV = requests.get(url_IV).json()

In [27]:
venues_IV = results_IV['response']['groups'][0]['items']
    
nearby_venues_IV = json_normalize(venues_IV) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_IV =nearby_venues_IV.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_IV['venue.categories'] = nearby_venues_IV.apply(get_category_type, axis=1)

# clean columns
nearby_venues_IV.columns = [col.split(".")[-1] for col in nearby_venues_IV.columns]

nearby_venues_IV.head()



,name,categories,lat,lng
0,Roadhouse,American Restaurant,41.933792,12.603205
1,Old Wild West,American Restaurant,41.933149,12.589207
2,Gattobianco Bar-Tabacchi-Tavola Calda,Food Court,41.932602,12.585009
3,Lidl,Supermarket,41.926983,12.574954
4,Reginè Tiburtina,Pizza Place,41.930148,12.577782


Search for only Pizza places or Italian restaurants

In [28]:
Venues_IV = nearby_venues_IV.loc[nearby_venues_IV['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_IV

,name,categories,lat,lng
4,Reginè Tiburtina,Pizza Place,41.930148,12.577782
11,Ristocenter,Italian Restaurant,41.928053,12.608141
12,AZZURRA Pizzeria Bisteccheria,Pizza Place,41.927474,12.574820


Find fifth neighborhood

In [29]:
neighborhood_latitude_V = merged_data.loc[4, 'Latitude']
neighborhood_longitude_V = merged_data.loc[4, 'Longitude']
neighborhood_name_V = merged_data.loc[4, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_V, neighborhood_latitude_V, neighborhood_longitude_V))

Latitude and longitude values of Prenestino/Centocelle are 41.888459999999995, 12.57574


Get top 100 venues in **Prenestino/Centocelle** within radius of 1500 meters

In [30]:
LIMIT = 100
radius = 1500
url_V = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_V, 
    neighborhood_longitude_V, 
    radius, 
    LIMIT)
url_V

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.888459999999995,12.57574&radius=1500&limit=100'

In [31]:
results_V = requests.get(url_V).json()

In [32]:
venues_V = results_V['response']['groups'][0]['items']
    
nearby_venues_V = json_normalize(venues_V) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_V =nearby_venues_V.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_V['venue.categories'] = nearby_venues_V.apply(get_category_type, axis=1)

# clean columns
nearby_venues_V.columns = [col.split(".")[-1] for col in nearby_venues_V.columns]

nearby_venues_V.head()


,name,categories,lat,lng
0,CSOA Forte Prenestino,Music Venue,41.892687,12.568937
1,Ristorante Cinese Gui Lin,Chinese Restaurant,41.882570,12.568630
2,OmbraLonga,Wine Bar,41.889561,12.566099
3,Ristorante Il Coccio,Italian Restaurant,41.881127,12.577022
4,Cornetteria di Centocelle,Dessert Shop,41.889210,12.570698


Search for only Pizza places or Italian restaurants

In [33]:
Venues_V = nearby_venues_V.loc[nearby_venues_V['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_V

,name,categories,lat,lng
3,Ristorante Il Coccio,Italian Restaurant,41.881127,12.577022
9,The Fooders - Mazzo,Italian Restaurant,41.889908,12.563321
11,Alice Pizza Prenestina,Pizza Place,41.894894,12.560811
13,Buone Maniere,Pizza Place,41.879462,12.563528
19,Scaramouche,Italian Restaurant,41.897471,12.564638
20,Pizzeria Da Marco,Pizza Place,41.896628,12.566520
21,Quagliaro,Italian Restaurant,41.893469,12.576825
28,Paperi e Papere,Pizza Place,41.889483,12.561233
31,Domus Pinsa,Pizza Place,41.894224,12.591525
33,Mangianapoli,Pizza Place,41.891122,12.558024


Find sixth neighborhood

In [34]:
neighborhood_latitude_VI = merged_data.loc[5, 'Latitude']
neighborhood_longitude_VI = merged_data.loc[5, 'Longitude']
neighborhood_name_VI = merged_data.loc[5, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_VI, neighborhood_latitude_VI, neighborhood_longitude_VI))

Latitude and longitude values of Roma Delle Torri are 41.8869, 12.687289999999999


Get top 100 venues in **Roma Delle Torri** within radius of 1500 meters

In [35]:
LIMIT = 100
radius = 1500
url_VI = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_VI, 
    neighborhood_longitude_VI, 
    radius, 
    LIMIT)
url_VI

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.8869,12.687289999999999&radius=1500&limit=100'

In [36]:
results_VI = requests.get(url_VI).json()

In [37]:
venues_VI = results_VI['response']['groups'][0]['items']
    
nearby_venues_VI = json_normalize(venues_VI) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_VI =nearby_venues_VI.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_VI['venue.categories'] = nearby_venues_VI.apply(get_category_type, axis=1)

# clean columns
nearby_venues_VI.columns = [col.split(".")[-1] for col in nearby_venues_VI.columns]

nearby_venues_VI.head()


,name,categories,lat,lng
0,Heaven 3,Gym,41.892457,12.675069
1,Centro Commerciale Unico,Boutique,41.894718,12.673087
2,Mauris,Shop & Service,41.891390,12.679900
3,GdC Ponte Di Nona,Soccer Field,41.894284,12.677151
4,SCHIAVI WALTER,Automotive Shop,41.880803,12.672377


Search for only Pizza places or Italian restaurants

In [38]:
Venues_VI = nearby_venues_VI.loc[nearby_venues_VI['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_VI

,name,categories,lat,lng


Find seventh neighborhood

In [39]:
neighborhood_latitude_VII = merged_data.loc[6, 'Latitude']
neighborhood_longitude_VII = merged_data.loc[6, 'Longitude']
neighborhood_name_VII = merged_data.loc[6, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_VII, neighborhood_latitude_VII, neighborhood_longitude_VII))

Latitude and longitude values of Appio-Latino/Tuscolano/Cinecittà are 41.8384, 12.58207


Get top 100 venues in **Appio-Latino/Tuscolano/Cinecittà** within radius of 1500 meters

In [40]:
LIMIT = 100
radius = 1500
url_VII = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_VII, 
    neighborhood_longitude_VII, 
    radius, 
    LIMIT)
url_VII

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.8384,12.58207&radius=1500&limit=100'

In [41]:
results_VII = requests.get(url_VII).json()

In [42]:
venues_VII = results_VII['response']['groups'][0]['items']
    
nearby_venues_VII = json_normalize(venues_VII) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_VII =nearby_venues_VII.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_VII['venue.categories'] = nearby_venues_VII.apply(get_category_type, axis=1)

# clean columns
nearby_venues_VII.columns = [col.split(".")[-1] for col in nearby_venues_VII.columns]

nearby_venues_VII.head()


,name,categories,lat,lng
0,Cinemusic Hotel,Hotel,41.843860,12.583050
1,IKEA Roma Anagnina,Furniture / Home Store,41.835662,12.593249
2,Hop & Pork,Brewery,41.847714,12.591199
3,Casale Consolini,Italian Restaurant,41.842164,12.572803
4,McDonald's,Fast Food Restaurant,41.838336,12.593955


Search for only Pizza places or Italian restaurants

In [43]:
Venues_VII = nearby_venues_VII.loc[nearby_venues_VII['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_VII

,name,categories,lat,lng
3,Casale Consolini,Italian Restaurant,41.842164,12.572803
6,Ristorante Paradiso Terrestre,Italian Restaurant,41.835558,12.572487
9,L'Allegra Fattoria,Italian Restaurant,41.833271,12.568820
20,La Gazza,Pizza Place,41.839815,12.588854
25,Cottage,Pizza Place,41.844709,12.574947


Find eighth neighborhood

In [44]:
neighborhood_latitude_VIII = merged_data.loc[7, 'Latitude']
neighborhood_longitude_VIII = merged_data.loc[7, 'Longitude']
neighborhood_name_VIII = merged_data.loc[7, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_VIII, neighborhood_latitude_VIII, neighborhood_longitude_VIII))

Latitude and longitude values of Appia Antica are 41.82802, 12.52934


Get top 100 venues in **Appia Antica** within radius of 1500 meters

In [45]:
LIMIT = 100
radius = 1500
url_VIII = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_VIII, 
    neighborhood_longitude_VIII, 
    radius, 
    LIMIT)
url_VIII

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.82802,12.52934&radius=1500&limit=100'

In [46]:
results_VIII = requests.get(url_VIII).json()

In [47]:
venues_VIII = results_VIII['response']['groups'][0]['items']
    
nearby_venues_VIII = json_normalize(venues_VIII) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_VIII =nearby_venues_VIII.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_VIII['venue.categories'] = nearby_venues_VIII.apply(get_category_type, axis=1)

# clean columns
nearby_venues_VIII.columns = [col.split(".")[-1] for col in nearby_venues_VIII.columns]

nearby_venues_VIII.head()


,name,categories,lat,lng
0,Circolo Tennis Oasi di Pace,Tennis Court,41.834175,12.533059
1,Centro Sportivo Le Magnolie,Pool,41.826502,12.527445
2,Dea di Roma,Pizza Place,41.821953,12.519190
3,Evodia Club,Soccer Field,41.827036,12.527632
4,Circolo Tennis Magnolie,Tennis Stadium,41.826491,12.527417


Search for only Pizza places or Italian restaurants

In [48]:
Venues_VIII = nearby_venues_VIII.loc[nearby_venues_VIII['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_VIII

,name,categories,lat,lng
2,Dea di Roma,Pizza Place,41.821953,12.51919


Find ninth neighborhood

In [49]:
neighborhood_latitude_IX = merged_data.loc[8, 'Latitude']
neighborhood_longitude_IX = merged_data.loc[8, 'Longitude']
neighborhood_name_IX = merged_data.loc[8, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_IX, neighborhood_latitude_IX, neighborhood_longitude_IX))

Latitude and longitude values of EUR are 41.757, 12.49799


Get top 100 venues in **EUR** within radius of 1500 meters

In [50]:
LIMIT = 100
radius = 1500
url_IX = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_IX, 
    neighborhood_longitude_IX, 
    radius, 
    LIMIT)
url_IX

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.757,12.49799&radius=1500&limit=100'

In [51]:
results_IX = requests.get(url_IX).json()

In [52]:
venues_IX = results_IX['response']['groups'][0]['items']
    
nearby_venues_IX = json_normalize(venues_IX) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_IX =nearby_venues_IX.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_IX['venue.categories'] = nearby_venues_IX.apply(get_category_type, axis=1)

# clean columns
nearby_venues_IX.columns = [col.split(".")[-1] for col in nearby_venues_IX.columns]

nearby_venues_IX.head()


,name,categories,lat,lng
0,Zampetta,Pet Store,41.751607,12.501015
1,laghi di trigoria,Lake,41.749958,12.482972
2,Belvedere Giovanni XXIII,Scenic Lookout,41.762611,12.481611


Search for only Pizza places or Italian restaurants

In [53]:
Venues_IX = nearby_venues_IX.loc[nearby_venues_IX['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_IX

,name,categories,lat,lng


Find tenth neighborhood

In [54]:
neighborhood_latitude_X = merged_data.loc[9, 'Latitude']
neighborhood_longitude_X = merged_data.loc[9, 'Longitude']
neighborhood_name_X = merged_data.loc[9, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_X, neighborhood_latitude_X, neighborhood_longitude_X))

Latitude and longitude values of Ostia/Acilia are 41.73875, 12.36328


Get top 100 venues in **Ostia/Acilia** within radius of 1500 meters

In [55]:
LIMIT = 100
radius = 1500
url_X = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_X, 
    neighborhood_longitude_X, 
    radius, 
    LIMIT)
url_X

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.73875,12.36328&radius=1500&limit=100'

In [56]:
results_X = requests.get(url_X).json()

In [57]:
venues_X = results_X['response']['groups'][0]['items']
    
nearby_venues_X = json_normalize(venues_X) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_X =nearby_venues_X.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_X['venue.categories'] = nearby_venues_X.apply(get_category_type, axis=1)

# clean columns
nearby_venues_X.columns = [col.split(".")[-1] for col in nearby_venues_X.columns]

nearby_venues_X.head()


,name,categories,lat,lng
0,jin xin yuan,Asian Restaurant,41.733009,12.368661
1,Carrefour Market,Supermarket,41.740265,12.370734
2,cinghiale,Italian Restaurant,41.731761,12.350988
3,Isola Verde,Pool,41.747098,12.355040
4,Old Wild West,Fast Food Restaurant,41.751656,12.367830


Search for only Pizza places or Italian restaurants

In [58]:
Venues_X = nearby_venues_X.loc[nearby_venues_X['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_X

,name,categories,lat,lng
2,cinghiale,Italian Restaurant,41.731761,12.350988
6,La Cucineria,Italian Restaurant,41.746590,12.363911
7,I GLADIATORI Osteria romana pizzeria,Italian Restaurant,41.748207,12.369010
9,Fraschetta Da Matteo,Italian Restaurant,41.735674,12.345803
16,Trattoria i Ciarli,Italian Restaurant,41.737081,12.374867
17,La Terrazza,Italian Restaurant,41.747214,12.354746


Find eleventh neighborhood

In [59]:
neighborhood_latitude_XI = merged_data.loc[10, 'Latitude']
neighborhood_longitude_XI = merged_data.loc[10, 'Longitude']
neighborhood_name_XI = merged_data.loc[10, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_XI, neighborhood_latitude_XI, neighborhood_longitude_XI))

Latitude and longitude values of Arvalia/Portuense are 41.830020000000005, 12.37696


Get top 100 venues in **Arvalia/Portuense** within radius of 1500 meters

In [60]:
LIMIT = 100
radius = 1500
url_XI = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_XI, 
    neighborhood_longitude_XI, 
    radius, 
    LIMIT)
url_XI

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.830020000000005,12.37696&radius=1500&limit=100'

In [61]:
results_XI = requests.get(url_XI).json()

In [62]:
venues_XI = results_XI['response']['groups'][0]['items']
    
nearby_venues_XI = json_normalize(venues_XI) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_XI =nearby_venues_XI.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_XI['venue.categories'] = nearby_venues_XI.apply(get_category_type, axis=1)

# clean columns
nearby_venues_XI.columns = [col.split(".")[-1] for col in nearby_venues_XI.columns]

nearby_venues_XI.head()


,name,categories,lat,lng
0,Executive Lounge Marriott Park,Hotel Bar,41.824698,12.391555
1,Rome Marriott Park Hotel,Hotel,41.824829,12.391934
2,Unieuro,Electronics Store,41.827237,12.386586
3,La Tedesca,Italian Restaurant,41.821790,12.374028
4,Emu Flag Store roma,Furniture / Home Store,41.824827,12.387354


Search for only Pizza places or Italian restaurants

In [63]:
Venues_XI = nearby_venues_XI.loc[nearby_venues_XI['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_XI

,name,categories,lat,lng
3,La Tedesca,Italian Restaurant,41.82179,12.374028


Find twelfth neighborhood

In [64]:
neighborhood_latitude_XII = merged_data.loc[11, 'Latitude']
neighborhood_longitude_XII = merged_data.loc[11, 'Longitude']
neighborhood_name_XII = merged_data.loc[11, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_XII, neighborhood_latitude_XII, neighborhood_longitude_XII))

Latitude and longitude values of Monte Verde are 41.87037, 12.34497


Get top 100 venues in **Monte Verde** within radius of 1500 meters

In [65]:
LIMIT = 100
radius = 1500
url_XII = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_XII, 
    neighborhood_longitude_XII, 
    radius, 
    LIMIT)
url_XII

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.87037,12.34497&radius=1500&limit=100'

In [66]:
results_XII = requests.get(url_XII).json()

In [67]:
venues_XII = results_XII['response']['groups'][0]['items']
    
nearby_venues_XII = json_normalize(venues_XII) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_XII =nearby_venues_XII.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_XII['venue.categories'] = nearby_venues_XII.apply(get_category_type, axis=1)

# clean columns
nearby_venues_XII.columns = [col.split(".")[-1] for col in nearby_venues_XII.columns]

nearby_venues_XII.head()


,name,categories,lat,lng
0,Exclusivevent Srl,Buffet,41.870967,12.355607
1,Parco Della Massimilla,Park,41.869085,12.356961
2,Mondo Convenienza,Furniture / Home Store,41.882414,12.351476
3,Good Wok Restaurant,Chinese Restaurant,41.881547,12.347733


Search for only Pizza places or Italian restaurants

In [68]:
Venues_XII = nearby_venues_XII.loc[nearby_venues_XII['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_XII

,name,categories,lat,lng


Find thirteenth neighborhood

In [69]:
neighborhood_latitude_XIII = merged_data.loc[12, 'Latitude']
neighborhood_longitude_XIII = merged_data.loc[12, 'Longitude']
neighborhood_name_XIII = merged_data.loc[12, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_XIII, neighborhood_latitude_XIII, neighborhood_longitude_XIII))

Latitude and longitude values of Aurelia are 41.90979, 12.3463


Get top 100 venues in **Aurelia** within radius of 1500 meters

In [70]:
LIMIT = 100
radius = 1500
url_XIII = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_XIII, 
    neighborhood_longitude_XIII, 
    radius, 
    LIMIT)
url_XIII

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.90979,12.3463&radius=1500&limit=100'

In [71]:
results_XIII = requests.get(url_XIII).json()

In [72]:
venues_XIII = results_XIII['response']['groups'][0]['items']
    
nearby_venues_XIII = json_normalize(venues_XIII) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_XIII =nearby_venues_XIII.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_XIII['venue.categories'] = nearby_venues_XIII.apply(get_category_type, axis=1)

# clean columns
nearby_venues_XIII.columns = [col.split(".")[-1] for col in nearby_venues_XIII.columns]

nearby_venues_XIII.head()


,name,categories,lat,lng
0,la collina storta,Stables,41.907634,12.341379
1,CD Removals Traslochi,Home Service,41.897450,12.345730


Search for only Pizza places or Italian restaurants

In [73]:
Venues_XIII = nearby_venues_XIII.loc[nearby_venues_XIII['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_XIII

,name,categories,lat,lng


Find fourteenth neighborhood

In [74]:
neighborhood_latitude_XIV = merged_data.loc[13, 'Latitude']
neighborhood_longitude_XIV = merged_data.loc[13, 'Longitude']
neighborhood_name_XIV = merged_data.loc[13, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_XIV, neighborhood_latitude_XIV, neighborhood_longitude_XIV))

Latitude and longitude values of Monte Mario are 41.97501, 12.34567


Get top 100 venues in **Monte Mario** within radius of 1500 meters

In [75]:
LIMIT = 100
radius = 1500
url_XIV = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_XIV, 
    neighborhood_longitude_XIV, 
    radius, 
    LIMIT)
url_XIV

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=41.97501,12.34567&radius=1500&limit=100'

In [76]:
results_XIV = requests.get(url_XIV).json()

In [77]:
venues_XIV = results_XIV['response']['groups'][0]['items']
    
nearby_venues_XIV = json_normalize(venues_XIV) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_XIV =nearby_venues_XIV.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_XIV['venue.categories'] = nearby_venues_XIV.apply(get_category_type, axis=1)

# clean columns
nearby_venues_XIV.columns = [col.split(".")[-1] for col in nearby_venues_XIV.columns]

nearby_venues_XIV.head()


,name,categories,lat,lng
0,Poggio Dei Cavalieri,Restaurant,41.978386,12.340647
1,Casale San Nicola,Hotel,41.963683,12.354047


Search for only Pizza places or Italian restaurants

In [78]:
Venues_XIV = nearby_venues_XIV.loc[nearby_venues_XIV['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_XIV

,name,categories,lat,lng


Find fifteenth neighborhood

In [79]:
neighborhood_latitude_XV = merged_data.loc[14, 'Latitude']
neighborhood_longitude_XV = merged_data.loc[14, 'Longitude']
neighborhood_name_XV = merged_data.loc[14, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}'.format(neighborhood_name_XV, neighborhood_latitude_XV, neighborhood_longitude_XV))

Latitude and longitude values of Cassia/Flaminia are 42.029070000000004, 12.41586


Get top 100 venues in **Cassia/Flaminia** within radius of 1500 meters

In [80]:
LIMIT = 100
radius = 1500
url_XV = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_XV, 
    neighborhood_longitude_XV, 
    radius, 
    LIMIT)
url_XV

'https://api.foursquare.com/v2/venues/explore?&client_id=ERP4VLV1V2S50EPBVX0WP1PSBSKGF3AHZBIXU5CGOYS3LMEZ&client_secret=Z5NGERSC4UNC1ZZUB0SDGUY11OZ1GQ1M3AMJQCQ02RQIB4S4&v=20180605&ll=42.029070000000004,12.41586&radius=1500&limit=100'

In [81]:
results_XV = requests.get(url_XV).json()

In [82]:
venues_XV = results_XV['response']['groups'][0]['items']
    
nearby_venues_XV = json_normalize(venues_XV) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_XV =nearby_venues_XV.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_XV['venue.categories'] = nearby_venues_XV.apply(get_category_type, axis=1)

# clean columns
nearby_venues_XV.columns = [col.split(".")[-1] for col in nearby_venues_XV.columns]

nearby_venues_XV.head()


,name,categories,lat,lng
0,Le 7 Fonti,Italian Restaurant,42.027234,12.411190
1,Parco natura La selvotta,Park,42.037069,12.417157
2,Momoe a.c.,Restaurant,42.039450,12.404649


Search for only Pizza places or Italian restaurants

In [83]:
Venues_XV = nearby_venues_XV.loc[nearby_venues_XV['categories'].isin(['Pizza Place','Italian Restaurant'])]
Venues_XV

,name,categories,lat,lng
0,Le 7 Fonti,Italian Restaurant,42.027234,12.41119


### Step 2: Combine data from step 1 into new dataframe

Create dataframes for each municipio containing columns: "Municipio", "nr of Italian Restaurants" and "nr of Pizza Places"

In [84]:
# create dataframe per municipio with total of selected venues from data above
municipio_I = Venues_I.groupby('categories').count()[['name']]
municipio_II = Venues_II.groupby('categories').count()[['name']]
municipio_III = Venues_III.groupby('categories').count()[['name']]
municipio_IV = Venues_IV.groupby('categories').count()[['name']]
municipio_V = Venues_V.groupby('categories').count()[['name']]
municipio_VI = Venues_VI.groupby('categories').count()[['name']]
municipio_VII = Venues_VII.groupby('categories').count()[['name']]
municipio_VIII = Venues_VIII.groupby('categories').count()[['name']]
municipio_IX = Venues_IX.groupby('categories').count()[['name']]
municipio_X = Venues_X.groupby('categories').count()[['name']]
municipio_XI = Venues_XI.groupby('categories').count()[['name']]
municipio_XII = Venues_XII.groupby('categories').count()[['name']]
municipio_XIII = Venues_XIII.groupby('categories').count()[['name']]
municipio_XIV = Venues_XIV.groupby('categories').count()[['name']]
municipio_XV = Venues_XV.groupby('categories').count()[['name']]

# change 'name' to number of municipio
municipio_I.rename(columns = {'name':'I'}, inplace=True)
municipio_II.rename(columns = {'name':'II'}, inplace=True)
municipio_III.rename(columns = {'name':'III'}, inplace=True)
municipio_IV.rename(columns = {'name':'IV'}, inplace=True)
municipio_V.rename(columns = {'name':'V'}, inplace=True)
municipio_VI.rename(columns = {'name':'VI'}, inplace=True)
municipio_VII.rename(columns = {'name':'VII'}, inplace=True)
municipio_VIII.rename(columns = {'name':'VIII'}, inplace=True)
municipio_IX.rename(columns = {'name':'IX'}, inplace=True)
municipio_X.rename(columns = {'name':'X'}, inplace=True)
municipio_XI.rename(columns = {'name':'XI'}, inplace=True)
municipio_XII.rename(columns = {'name':'XII'}, inplace=True)
municipio_XIII.rename(columns = {'name':'XIII'}, inplace=True)
municipio_XIV.rename(columns = {'name':'XIV'}, inplace=True)
municipio_XV.rename(columns = {'name':'XV'}, inplace=True)


In [85]:
# transpose dataframes
municipio_I = municipio_I.T
municipio_II = municipio_II.T
municipio_III = municipio_III.T
municipio_IV = municipio_IV.T
municipio_V = municipio_V.T
municipio_VI = municipio_VI.T
municipio_VII = municipio_VII.T
municipio_VIII = municipio_VIII.T
municipio_IX = municipio_IX.T
municipio_X = municipio_X.T
municipio_XI = municipio_XI.T
municipio_XII = municipio_XII.T
municipio_XIII = municipio_XIII.T
municipio_XIV = municipio_XIV.T
municipio_XV = municipio_XV.T



In [86]:
# merge new dataframes into one dataframe
all_municipio = municipio_I.append([municipio_II,municipio_III,municipio_IV,municipio_V,municipio_VI,municipio_VII,municipio_VIII,municipio_IX,municipio_X,municipio_XI,municipio_XII,municipio_XIII,municipio_XIV,municipio_XV], sort=True)


# reset index and rename columns to appropriate titles
all_municipio.index.names = ['Municipio']
all_municipio.rename(columns = {'Italian Restaurant': 'nr of Italian restaurants', 'Pizza Place': 'nr of pizza places'}, inplace=True)
all_municipio = all_municipio.reset_index()

# pre-process values 'NaN'
all_municipio = all_municipio.fillna(0)

# add column with sum of Italian restaurants and pizza places per municipio
all_municipio['total'] = all_municipio['nr of Italian restaurants'] + all_municipio['nr of pizza places']



Merge latest dataframe with dataframe containing population and density info

In [87]:
rome_data = pd.merge(df_data, all_municipio)


Sort dataframe by total of venues (in descending order) and by population density

In [88]:
rome_data.sort_values(by=['total', 'Density'], ascending=False, inplace=True)

#replace index with Municipio
rome_data.set_index('Municipio', inplace=True)

rome_data

,Neighborhood,Population,Area,Density,nr of Italian restaurants,nr of pizza places,total
Municipio,,,,,,,
II,Parioli/Nomentano,167736,19.60,8567,17.0,8.0,25.0
I,Historical Center,186802,19.91,9382,11.0,5.0,16.0
V,Prenestino/Centocelle,246471,27.00,9137,5.0,8.0,13.0
X,Ostia/Acilia,230544,150.64,1530,6.0,0.0,6.0
VII,Appio-Latino/Tuscolano/Cinecittà,307607,46.80,6580,3.0,2.0,5.0
IV,Tiburtina,177084,49.15,3603,1.0,2.0,3.0
VIII,Appia Antica,131082,47.29,2772,0.0,1.0,1.0
XI,Arvalia/Portuense,154871,70.90,2185,1.0,0.0,1.0
XV,Cassia/Flaminia,158561,186.70,849,1.0,0.0,1.0


## Results

In the final data frame (rome_data), one can see that there are still municipio/subdivisions in the city of Rome, Italy where no Italian restaurants or pizza places are situated. Therefore, the assumption that there are pizzerias on about every corner of every busy street is not true. 

As one can also see in the ‘nearby_venues’-data frames per municipio, there are even still areas without any restaurants (apart from hotels maybe). For example municipio III: ‘Monte Sacro’. When searching for the top 100 venues, only 4 were returned by the FourSquare API, of which one was a hotel, but the rest was ‘pool’, ‘farm’ and ‘mobile phone shop’. I will dive further into this and the location / population density in the ‘Discussion’-section of this report.


## Discussion

This project aimed to find out whether there is still room for a new pizzeria in the city of Rome, Italy and, if that were the case, where the competition is the least ‘killing’ and people are still willing to try out his pizzas. In order to answer these questions, this section is divided in several smaller sections.

### Is there still room for a new pizzeria?

Investigation focussed on this question, comprised of evaluating whether there are still areas in Rome where there are few or no pizzerias or Italian restaurants. The latter was included because these restaurants often have pizzas on their menus. All other venues were excluded from this research. The search for postal code information on Rome led to a Wikipedia-page where fifteen administrative subdivisions or “Municipio” of Rome were described (see figure below). 

![Image of municipio Rome](https://upload.wikimedia.org/wikipedia/commons/e/eb/Roma_-_Municipi_numerata.png)

*Picture of the locations of the municipio in Rome*

Geonames.org used the same Municipio and was helpful to provide the latitude and longitude per subdivision. With this information, a map of Rome was created with a range of 1500 meters around the coordinates of each Municipio (“map_rome”; already shown in the Methods-section), showing in which areas the next step was performed: calling the FourSquare API to find the top 100 venues within each area. (Top 1000 of venues was also performed, but the API did not exceed past 100 venues.)


With the information of the found venues, a selection was made to only include the categories ‘Pizza place’ and ‘Italian restaurant’. A data frame was created combining the relevant results of all Municipio and a total of relevant venues was calculated. This final data frame “rome_data” showed that six out of fifteen subdivisions of Rome did not have a relevant venue, and 3 had only 1 (either a pizza place or an Italian restaurant). This answers the first question: yes, there is still room for a new pizzeria.


### Where would be the best place for a new pizzeria?

As previously stated, there are still areas in Rome with no (or very few) pizza selling restaurant(s). Additional information about these areas (such as population, area in km2, and density in km2) were also provided in the final data frame “rome_data”. The most dense Municipio (as in, having the largest population per square kilometre) already own more than three places selling pizzas. These areas were Historical Center, Parioli/Nomentano, Prenestino/Centocelle, Ostia/Acilia, Appio-Latino/Tuscolano/Cinecittà and Tiburtina. I would recommend staying away from those Municipio as a new starting pizza place/restaurant. However, subdivisions like Monte Sacro having a density of 2091 people per square kilometre, might be a very nice place to start. Although a bit far from the city centre (and probably the most tourist), its citizens might be craving for a pizza. Another example: Municipio VIII or “Appia Antica” is relatively close to the centre of Rome and also doesn’t own a pizza selling place. Its density is not far from that of Monte Sacro (1949 people per square kilometre).

Answering this question can be done in three manners: (1) choosing the closest area to the city centre with the least competition, or (2) choosing the area with the largest density and the least competition. Finally, (3) one can take the touristic places into account when picking the most popular place out of those without any pizza selling places. This latter method might seem the same as the first, however, there are touristic places that are not necessarily in or around the historical city centre. Within this research, though, I will only look at method 1 and 2, since that can be answered with the provided information within the Method-section. Tourism of a city is a whole other aspect.

So, using method 1 (closest area to city centre), I would recommend Municipio VIII / “Appia Antica” as an area to start a new pizzeria. Although owning 1 Italian restaurant, the addition of a pizzeria might be profitable. 

Using method 2 (choosing area with largest density), Appia Antica also comes up as the best place in town (2772 people per square kilometre, with 1 competitor), with Roma Delle Tori (municipio VI) as first runner up with 2261 people per square kilometre and zero competitors.


### Points for improvement for future research on this topic

Within this project, a 1500 meter range was drawn around the coordinates of each municipio (as was shown in "map_rome"), making sure no areas overlapped with each other. This range excluded all possible venues that were situated within the area, but outside of the 1500 meter range. For future research, it could be more useful to find a way to include all venues within the complete municipio.
In addition, the FourSquare API returned only the top 100 venues per input. It is possible that within the search areas, there are more venues that were not included in this project.
Finally, touristic information was not included within this research as is was ‘out of scope’. In order to investigate whether the third method described in the section above leads to another result/recommendation than the first two, this will be useful.

## Conclusion

After investigating whether there is still room for a new pizzeria in Rome, Italy, and if so, where the best place would be, both described methods in the ‘Discussion’-section lead to the same ‘winning’ area. Appia Antica is the municipio closest to the city centre with only one competitor and relatively many people per square kilometre. Therefore, this would be the area I recommend for a new pizzeria.